<a href="https://colab.research.google.com/github/ZahinAwosaf/NLP-Projects/blob/main/character_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import random
import time

import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers, utils, Sequential
from keras.layers import *

In [25]:
np.seterr(divide = 'ignore') 

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
path = tf.keras.utils.get_file("Alice.txt", "https://raw.githubusercontent.com/ZahinAwosaf/NLP-Projects/main/data/Alice.txt")

163840/147673 [=================================] - 0s 0us/step


In [3]:
text = open(path, "rb").read().decode(encoding = "utf-8")

In [4]:
text = text.replace("\n", " ")
text_length = len(text)
print("Length Text:", text_length)

Length Text: 147673


In [5]:
print(text[:250])

 book her sister was reading, but it had no pictures or conversations i


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Total unique characters:", vocab_size)

Total unique characters: 69


In [7]:
chars_to_ids = {u:i for i, u in enumerate(chars)}
ids_to_chars = {i:u for i, u in enumerate(chars)}

In [8]:
max_length = 50
step = 3
sentences = []
next_chars = []

for i in range(0, text_length - max_length, step):
    sentences.append(text[i : i + max_length])
    next_chars.append(text[i + max_length])

print("Number of sequences:", len(sentences))

Number of sequences: 49208


In [9]:
X = np.zeros((len(sentences), max_length, vocab_size), dtype = np.bool)
y = np.zeros((len(sentences), vocab_size), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        X[i, j, chars_to_ids[char]] = 1
    y[i, chars_to_ids[next_chars[i]]] = 1

In [10]:
units = 128
batch_size = 256

In [11]:
model = Sequential([
    Input(shape=(max_length, vocab_size)),
    LSTM(units),
    Dense(vocab_size, activation = "softmax")
])

model.compile(loss = keras.losses.CategoricalCrossentropy(),
              optimizer = keras.optimizers.RMSprop(learning_rate = 0.01))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               101376    
                                                                 
 dense (Dense)               (None, 69)                8901      
                                                                 
Total params: 110,277
Trainable params: 110,277
Non-trainable params: 0
_________________________________________________________________


In [12]:
def text_sampling_function(predictions, diversity = 1.0):
    predictions = tf.cast(np.asarray(predictions), dtype = tf.float64)
    predictions = np.log(predictions) / diversity
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    probabilities = np.random.multinomial(1, predictions, 1)
    
    return np.argmax(probabilities)

In [13]:
model.fit(X, y, batch_size = batch_size, epochs = 300)

Epoch 1/300
193/193 [==============================] - 10s 25ms/step - loss: 2.4994
Epoch 2/300
193/193 [==============================] - 5s 26ms/step - loss: 1.8964
Epoch 3/300
193/193 [==============================] - 5s 26ms/step - loss: 1.6842
Epoch 4/300
193/193 [==============================] - 5s 25ms/step - loss: 1.5487
Epoch 5/300
193/193 [==============================] - 5s 25ms/step - loss: 1.4416
Epoch 6/300
193/193 [==============================] - 5s 26ms/step - loss: 1.3540
Epoch 7/300
193/193 [==============================] - 5s 26ms/step - loss: 1.2843
Epoch 8/300
193/193 [==============================] - 5s 26ms/step - loss: 1.2224
Epoch 9/300
193/193 [==============================] - 5s 26ms/step - loss: 1.1660
Epoch 10/300
193/193 [==============================] - 5s 26ms/step - loss: 1.1234
Epoch 11/300
193/193 [==============================] - 5s 26ms/step - loss: 1.0866
Epoch 12/300
193/193 [==============================] - 5s 26ms/step - loss: 1.0488


In [14]:
def generate_text(model, diversities, 
                  generate, start_string):
    
    for diversity in diversities:
        print()
        print("Diversity:", diversity)

        generated = ""
        sentence = start_string
        print("Starting String:", start_string)

        for i in range(generate):
            X_pred = np.zeros((1, max_length, vocab_size))
            for j, char in enumerate(sentence):
                X_pred[0, j, chars_to_ids[char]] = 1.0
            
            predictions = model.predict(X_pred, verbose = 0)[0]
            next_index = text_sampling_function(predictions, diversity)
            next_char = ids_to_chars[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("Generated:", generated)
        print()

In [40]:
generate_text(model, diversities = [0.2, 0.5, 1.0], generate = 100,
                    start_string = "In another moment down went Alice after it, never ")


Diversity: 0.2
Starting String: In another moment down went Alice after it, never 
 a Gry now and the were three could not at


Diversity: 0.5
Starting String: In another moment down went Alice after it, never 
 when she had someted her head wondent wordend, which was at the 


Diversity: 1.0
Starting String: In another moment down went Alice after it, never 
Generated: over the miling under that she next would she pupting on by try and no did nosered on the song, wond

